https://dacon.io/competitions/open/235539/overview/description

타이타닉은 역사상 가장 유명한 난파선 중 하나입니다.

1912년 4월 15일, 타이타닉은 항해중 빙산과의 충돌로 인해 침몰합니다. 탑승인원 2224명 중 1502명이 사망한 비극적인 사건이었고, 선박의 안전규정을 개선시키는 계기가 되었습니다.

이 과제에서는 여러분은 어떤 종류의 사람들이 많이 살아남았는지에 대한 분석을 해야 합니다.
그리고 머신러닝을 이용해 어떤 승객이 생존했을지 예측해보세요. 

1. train.csv / test.csv : 타이타닉 탑승자들 중 일부의 인적 정보와 생존 여부 데이터
PassengerID : 탑승객 고유 아이디
Survival : 탑승객 생존 유무 (0: 사망, 1: 생존)
Pclass : 등실의 등급
Name : 이름
Sex : 성별
Age : 나이
Sibsp : 함께 탐승한 형제자매, 아내, 남편의 수
Parch : 함께 탐승한 부모, 자식의 수
Ticket :티켓 번호
Fare : 티켓의 요금
Cabin : 객실번호
Embarked : 배에 탑승한 항구 이름 ( C = Cherbourn, Q = Queenstown, S = Southampton)

2. sample_submission.csv : 정답 파일의 예시

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.linear_model    import LogisticRegression
from sklearn.linear_model    import SGDClassifier
from sklearn.metrics         import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

In [2]:
path = './dataset/타이타닉/'

In [51]:
df_train = pd.read_csv(path + 'train.csv')
df_test = pd.read_csv(path + 'test.csv')
df_submission = pd.read_csv(path + 'submission.csv')

In [66]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [53]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [54]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## Data preprocessing

In [55]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [56]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


### - 상관성이 없다고 판단되는 Name 제거

In [57]:
df_train.drop(columns=['Name'], inplace= True)
df_test.drop(columns=['Name'], inplace= True)

In [58]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


### - Ticket 데이터 형변환

In [67]:
df_train['Ticket'] = [i.split()[-1] for i in list(df_train['Ticket'])]
df_test['Ticket'] = [i.split()[-1] for i in list(df_test['Ticket'])]

In [12]:
df_train = df_train[(df_train['Ticket'] != 'LINE')]

In [13]:
df_train['Ticket'] = df_train['Ticket'].astype('int64')
df_test['Ticket'] = df_test['Ticket'].astype('int64')

In [68]:
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,female,NaN,1,2,6607,23.4500,NaN,S
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C


### - 레이블 인코딩 처리(Sex, Embarked)

In [15]:
df_train['Sex'] = np.where(df_train['Sex'] == 'male', 1 , 0)
df_test['Sex'] = np.where(df_test['Sex'] == 'male', 1 , 0)

In [16]:
df_train['Cabin'].fillna('n', inplace= True)
df_test['Cabin'].fillna('n', inplace= True)

In [17]:
df_train['Cabin'] = df_train['Cabin'].str[:1]
df_test['Cabin'] = df_test['Cabin'].str[:1]

In [18]:
feature = ['Cabin', 'Embarked']

for i in feature:
    le = LabelEncoder()
    le = le.fit(df_train[i])
    df_train[i] = le.transform(df_train[i])
    df_test[i] = le.transform(df_test[i])

### - 상관 계수 확인

In [45]:
df_train.corr(method='pearson')

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,1.000000,-0.004948,-0.033674,0.044442,0.035636,-0.058525,-0.002551,-0.022994,0.011458,-0.030008,0.014209
Survived,-0.004948,1.000000,-0.338637,-0.544165,-0.075807,-0.036000,0.081241,-0.096400,0.257248,-0.301241,-0.163238
Pclass,-0.033674,-0.338637,1.000000,0.129507,-0.371676,0.085026,0.020252,0.286279,-0.548447,0.746279,0.155302
Sex,0.044442,-0.544165,0.129507,1.000000,0.092822,-0.113249,-0.244337,0.077330,-0.180568,0.121563,0.102327
Age,0.035636,-0.075807,-0.371676,0.092822,1.000000,-0.308528,-0.189100,-0.107166,0.097011,-0.268562,-0.025826
SibSp,-0.058525,-0.036000,0.085026,-0.113249,-0.308528,1.000000,0.414244,0.046018,0.158494,0.042705,0.067984
Parch,-0.002551,0.081241,0.020252,-0.244337,-0.189100,0.414244,1.000000,-0.033529,0.215153,-0.031468,0.039614
Ticket,-0.022994,-0.096400,0.286279,0.077330,-0.107166,0.046018,-0.033529,1.000000,-0.156916,0.190335,0.192277
Fare,0.011458,0.257248,-0.548447,-0.180568,0.097011,0.158494,0.215153,-0.156916,1.000000,-0.522303,-0.219903
Cabin,-0.030008,-0.301241,0.746279,0.121563,-0.268562,0.042705,-0.031468,0.190335,-0.522303,1.000000,0.185903


- 상관 계수를 확인해볼 때 생존 유무와 연관성이 짙은 feature는 Pclass(등실의 등급), Sex(성별), Fare(티켓의 요금), Embarked(배에 탑승한 항구 이름)으로 나타난다.

## Train & Evaluate
- 0.1 이하 삭제

In [46]:
# x, y = df_train.loc[:, ['PassengerId', 'Pclass', 'Sex', 'SibSp', 'Parch',
#        'Ticket', 'Fare', 'Cabin', 'Embarked']], df_train.loc[:, 'Survived']  0.4104176210108414

x, y = df_train.loc[:, ['Pclass', 'Sex', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked']], df_train.loc[:, 'Survived']


In [50]:
stf = StratifiedKFold(n_splits=3, shuffle= True)

print('stf.get_n_splits  : ', stf.get_n_splits(x, y))

model_lr = LogisticRegression()

model_lr_dl = Sequential()
model_lr_dl.add(Dense(1, input_dim=x.shape[1], activation='sigmoid'))

sgd = optimizers.SGD(lr=0.01)
model_lr_dl.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['binary_accuracy'])


score = []
for train_idx, test_idx in stf.split(x, y):
    X_train, X_test = x.values[train_idx], x.values[test_idx]
    y_train, y_test = y.values[train_idx], y.values[test_idx]

    model_lr_dl.fit(X_train, y_train, epochs=300)
    pred_y = model_lr_dl.predict(X_test)
    
#     model_lr.fit(X_train, y_train)
#     pred_y = model_lr.predict(X_test)
    
    score.append(mean_squared_error(y_test, pred_y))
    

stf.get_n_splits  :  3
Epoch 1/300


C:\Users\HPE\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


19/19 [==============================] - 0s 1ms/step - loss: 141259696.0000 - binary_accuracy: 0.5854
Epoch 2/300
19/19 [==============================] - 0s 999us/step - loss: 82477648.0000 - binary_accuracy: 0.5685
Epoch 3/300
19/19 [==============================] - 0s 1ms/step - loss: 77814496.0000 - binary_accuracy: 0.5550
Epoch 4/300
19/19 [==============================] - 0s 1ms/step - loss: 118831088.0000 - binary_accuracy: 0.5330
Epoch 5/300
19/19 [==============================] - 0s 1ms/step - loss: 93666064.0000 - binary_accuracy: 0.5618
Epoch 6/300
19/19 [==============================] - 0s 1ms/step - loss: 117680176.0000 - binary_accuracy: 0.5465
Epoch 7/300
19/19 [==============================] - 0s 1ms/step - loss: 121858312.0000 - binary_accuracy: 0.5482
Epoch 8/300
19/19 [==============================] - 0s 1ms/step - loss: 92679600.0000 - binary_accuracy: 0.5770
Epoch 9/300
19/19 [==============================] - 0s 1ms/step - loss: 178622192.0000 - binary_accur

19/19 [==============================] - 0s 1ms/step - loss: 107035504.0000 - binary_accuracy: 0.5821
Epoch 73/300
19/19 [==============================] - 0s 1ms/step - loss: 148914768.0000 - binary_accuracy: 0.5635
Epoch 74/300
19/19 [==============================] - 0s 1ms/step - loss: 101208000.0000 - binary_accuracy: 0.5736
Epoch 75/300
19/19 [==============================] - 0s 1ms/step - loss: 144568752.0000 - binary_accuracy: 0.5195
Epoch 76/300
19/19 [==============================] - 0s 1ms/step - loss: 138353904.0000 - binary_accuracy: 0.5482
Epoch 77/300
19/19 [==============================] - 0s 999us/step - loss: 216315904.0000 - binary_accuracy: 0.5550
Epoch 78/300
19/19 [==============================] - 0s 1ms/step - loss: 111043256.0000 - binary_accuracy: 0.5939
Epoch 79/300
19/19 [==============================] - 0s 1ms/step - loss: 164484304.0000 - binary_accuracy: 0.5550
Epoch 80/300
19/19 [==============================] - 0s 999us/step - loss: 171240864.0000 

19/19 [==============================] - 0s 999us/step - loss: 119366616.0000 - binary_accuracy: 0.5415
Epoch 143/300
19/19 [==============================] - 0s 999us/step - loss: 70486224.0000 - binary_accuracy: 0.5584
Epoch 144/300
19/19 [==============================] - 0s 1ms/step - loss: 118528912.0000 - binary_accuracy: 0.5330
Epoch 145/300
19/19 [==============================] - 0s 1ms/step - loss: 210001456.0000 - binary_accuracy: 0.5364
Epoch 146/300
19/19 [==============================] - 0s 999us/step - loss: 123993768.0000 - binary_accuracy: 0.5635
Epoch 147/300
19/19 [==============================] - 0s 1ms/step - loss: 148547456.0000 - binary_accuracy: 0.5127
Epoch 148/300
19/19 [==============================] - 0s 999us/step - loss: 130572008.0000 - binary_accuracy: 0.5618
Epoch 149/300
19/19 [==============================] - 0s 999us/step - loss: 94122584.0000 - binary_accuracy: 0.5685
Epoch 150/300
19/19 [==============================] - 0s 999us/step - loss: 1

...

19/19 [==============================] - 0s 1ms/step - loss: 135794624.0000 - binary_accuracy: 0.5499
Epoch 263/300
19/19 [==============================] - 0s 1ms/step - loss: 147714976.0000 - binary_accuracy: 0.5448
Epoch 264/300
19/19 [==============================] - 0s 1ms/step - loss: 134045656.0000 - binary_accuracy: 0.5347
Epoch 265/300
19/19 [==============================] - 0s 1ms/step - loss: 75742248.0000 - binary_accuracy: 0.5753
Epoch 266/300
19/19 [==============================] - 0s 984us/step - loss: 111314856.0000 - binary_accuracy: 0.5482
Epoch 267/300
19/19 [==============================] - 0s 999us/step - loss: 111532776.0000 - binary_accuracy: 0.5415
Epoch 268/300
19/19 [==============================] - 0s 999us/step - loss: 179479136.0000 - binary_accuracy: 0.5635
Epoch 269/300
19/19 [==============================] - 0s 1ms/step - loss: 97422528.0000 - binary_accuracy: 0.5398
Epoch 270/300
19/19 [==============================] - 0s 1ms/step - loss: 9866224

19/19 [==============================] - 0s 999us/step - loss: 106646984.0000 - binary_accuracy: 0.5490
Epoch 33/300
19/19 [==============================] - 0s 1ms/step - loss: 119792224.0000 - binary_accuracy: 0.5101
Epoch 34/300
19/19 [==============================] - 0s 1ms/step - loss: 133842752.0000 - binary_accuracy: 0.5642
Epoch 35/300
19/19 [==============================] - 0s 999us/step - loss: 137446480.0000 - binary_accuracy: 0.5321
Epoch 36/300
19/19 [==============================] - 0s 1ms/step - loss: 102583168.0000 - binary_accuracy: 0.5270
Epoch 37/300
19/19 [==============================] - 0s 999us/step - loss: 95916544.0000 - binary_accuracy: 0.5118
Epoch 38/300
19/19 [==============================] - 0s 1ms/step - loss: 78938216.0000 - binary_accuracy: 0.5625
Epoch 39/300
19/19 [==============================] - 0s 944us/step - loss: 138124560.0000 - binary_accuracy: 0.5220
Epoch 40/300
19/19 [==============================] - 0s 999us/step - loss: 89075320.00

19/19 [==============================] - 0s 1ms/step - loss: 86125072.0000 - binary_accuracy: 0.5304
Epoch 104/300
19/19 [==============================] - 0s 1ms/step - loss: 94374280.0000 - binary_accuracy: 0.5304
Epoch 105/300
19/19 [==============================] - 0s 996us/step - loss: 183302896.0000 - binary_accuracy: 0.5456
Epoch 106/300
19/19 [==============================] - 0s 1ms/step - loss: 126120712.0000 - binary_accuracy: 0.5220
Epoch 107/300
19/19 [==============================] - 0s 1ms/step - loss: 163454144.0000 - binary_accuracy: 0.5253
Epoch 108/300
19/19 [==============================] - 0s 1000us/step - loss: 101363464.0000 - binary_accuracy: 0.5220
Epoch 109/300
19/19 [==============================] - 0s 1ms/step - loss: 129705600.0000 - binary_accuracy: 0.5524
Epoch 110/300
19/19 [==============================] - 0s 1ms/step - loss: 101590256.0000 - binary_accuracy: 0.5236
Epoch 111/300
19/19 [==============================] - 0s 1ms/step - loss: 86702096

19/19 [==============================] - 0s 999us/step - loss: 99027024.0000 - binary_accuracy: 0.4932
Epoch 173/300
19/19 [==============================] - 0s 999us/step - loss: 150710320.0000 - binary_accuracy: 0.5118
Epoch 174/300
19/19 [==============================] - 0s 1ms/step - loss: 122284240.0000 - binary_accuracy: 0.5507
Epoch 175/300
19/19 [==============================] - 0s 1ms/step - loss: 90519048.0000 - binary_accuracy: 0.5389
Epoch 176/300
19/19 [==============================] - 0s 999us/step - loss: 96389744.0000 - binary_accuracy: 0.5490
Epoch 177/300
19/19 [==============================] - 0s 1ms/step - loss: 91392248.0000 - binary_accuracy: 0.5456
Epoch 178/300
19/19 [==============================] - 0s 1ms/step - loss: 80137120.0000 - binary_accuracy: 0.5338
Epoch 179/300
19/19 [==============================] - 0s 1ms/step - loss: 119529392.0000 - binary_accuracy: 0.5355
Epoch 180/300
19/19 [==============================] - 0s 1ms/step - loss: 139000464.

19/19 [==============================] - 0s 999us/step - loss: 103387984.0000 - binary_accuracy: 0.5541
Epoch 243/300
19/19 [==============================] - 0s 1ms/step - loss: 92263232.0000 - binary_accuracy: 0.5608
Epoch 244/300
19/19 [==============================] - 0s 999us/step - loss: 111712024.0000 - binary_accuracy: 0.5473
Epoch 245/300
19/19 [==============================] - 0s 1ms/step - loss: 122387760.0000 - binary_accuracy: 0.5321
Epoch 246/300
19/19 [==============================] - 0s 1ms/step - loss: 98632944.0000 - binary_accuracy: 0.5152
Epoch 247/300
19/19 [==============================] - 0s 999us/step - loss: 125717280.0000 - binary_accuracy: 0.5389
Epoch 248/300
19/19 [==============================] - 0s 1ms/step - loss: 76516784.0000 - binary_accuracy: 0.5372
Epoch 249/300
19/19 [==============================] - 0s 1ms/step - loss: 122060216.0000 - binary_accuracy: 0.5253
Epoch 250/300
19/19 [==============================] - 0s 1ms/step - loss: 18716192

In [49]:
np.mean(score)
# score

0.38669644220491683

### - 상관 계수 0.1 이하 제거 후 다시 진행

In [ ]:
df_train.corr(method='pearson').iloc[:, 1]

In [ ]:
df_train_modify = df_train.drop(columns= ['PassengerId', 'Age', 'SibSp', 'Parch', 'Ticket'])

In [ ]:
df_train_modify.keys()

In [ ]:
x_1, y_1 = df_train_modify.loc[:, ['Pclass', 'Sex', 'Fare', 'Embarked']], df_train.loc[:, 'Survived']

In [ ]:
stf = StratifiedKFold(n_splits=5, shuffle= True)

print('stf.get_n_splits  : ', stf.get_n_splits(x, y))

model_lr = LogisticRegression()
model_sgd = SGDClassifier()
model_lr_dl = Sequential()
model_lr_dl.add(Dense(1, input_dim=1, activation='sigmoid'))

sgd = optimizers.SGD(lr=0.01)
model_lr_dl.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['binary_accuracy'])


score = []
for train_idx, test_idx in stf.split(x_1, y_1):
    X_train, X_test = x_1.values[train_idx], x_1.values[test_idx]
    y_train, y_test = y_1.values[train_idx], y_1.values[test_idx]

#     model_lr_dl.fit(X_train, y_train, epochs=200)
#     pred_y = model_lr_dl.predict(X_test)
    
    model_sgd.fit(X_train, y_train)
    pred_y = model_sgd.predict(X_test)
    
    score.append(mean_squared_error(y_test, pred_y))
    

In [ ]:
np.mean(score)

In [ ]:
df_train_modify.corr(method='pearson').iloc[:, 0]